In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import time
import pystan

C:\Users\natan\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [273]:
df=pd.read_csv('./mike_pence_tweets/with_financials/pence_twitter_data_with_dow_for_analysis.csv',parse_dates=['created_at'],index_col=0)

In [274]:
for x in df.columns:
    print(x)

text
Name
Screen Name
created_at
favorite_count
retweet_count
Language
Client
Tweet Type
Media Type
Media URLs
URLs
Hashtags
Mentions
trade_topic
china_topic
financial_topic
labor_topic
industry_topic
currency_rates_topic
general_related_topic
budget_topic
foreign_policy_topic
covid_topic
sensitive_topic
environment_topic
sentiment
month
1_minutes_forward_pct_change
5_minutes_forward_pct_change
10_minutes_forward_pct_change
15_minutes_forward_pct_change
30_minutes_forward_pct_change
1_hours_forward_pct_change
3_hours_forward_pct_change
1_minutes_forward_vol_sum
5_minutes_forward_vol_sum
10_minutes_forward_vol_sum
15_minutes_forward_vol_sum
30_minutes_forward_vol_sum
1_hours_forward_vol_sum


In [275]:
X=df.loc[:,['created_at',
'favorite_count',
'retweet_count',
'trade_topic',
'china_topic',
'financial_topic',
'labor_topic',
'industry_topic',
'currency_rates_topic',
'general_related_topic',
'budget_topic',
'foreign_policy_topic',
'covid_topic',
'sensitive_topic',
'environment_topic',
'sentiment',
'text'
]].copy()

In [276]:
y=df[['1_minutes_forward_pct_change',
'5_minutes_forward_pct_change',
'10_minutes_forward_pct_change',
'15_minutes_forward_pct_change',
'30_minutes_forward_pct_change',
'1_hours_forward_pct_change',
'1_minutes_forward_vol_sum',
'5_minutes_forward_vol_sum',
'10_minutes_forward_vol_sum',
'15_minutes_forward_vol_sum',
'30_minutes_forward_vol_sum',
'1_hours_forward_vol_sum']].copy()

In [278]:
inspect_df=pd.concat([X.loc[~y['5_minutes_forward_pct_change'].isna()],y['5_minutes_forward_pct_change'].dropna()],axis=1)

In [279]:
inspect_df=inspect_df.loc[inspect_df['created_at']>='2016-11-08 22:00:00'].copy()

In [280]:
for column in inspect_df.iloc[:,:-1].columns:
    inspect_df[column] = inspect_df[[column, 'sentiment']].product(axis=1)
# inspect_df.drop('sentiment',1,inplace=True)

In [281]:
inspect_df.drop(['created_at','text'],1,inplace=True)

In [282]:
# inspect_df=inspect_df.replace([np.inf, -np.inf], float('nan'))
inspect_df=inspect_df.dropna()

In [283]:
mod = sm.OLS(inspect_df.iloc[:,-1],sm.add_constant(inspect_df.iloc[:,:-1]), hasconst = False)
res = mod.fit()

In [284]:
print(res.summary())

                                 OLS Regression Results                                 
Dep. Variable:     5_minutes_forward_pct_change   R-squared:                       0.026
Model:                                      OLS   Adj. R-squared:                  0.001
Method:                           Least Squares   F-statistic:                     1.023
Date:                          Mon, 13 Jul 2020   Prob (F-statistic):              0.429
Time:                                  17:56:10   Log-Likelihood:                 3494.1
No. Observations:                           590   AIC:                            -6958.
Df Residuals:                               575   BIC:                            -6892.
Df Model:                                    15                                         
Covariance Type:                      nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------

# MCMC inspection

In [285]:
# Specify the model in Stan

model = """
data {
    int<lower=0> N;
    vector[N] x;
    vector[N] x1;
    vector[N] x2;
    vector[N] x3;
    vector[N] x4;
    vector[N] x5;
    vector[N] x6;
    vector[N] x7;
    vector[N] x8;
    vector[N] x9;
    vector[N] x10;
    vector[N] x11;
    vector[N] x12;
    vector[N] x13;
    vector[N] x14;
    vector[N] y;
}
parameters {
    real alpha;
    real favorite_count;
    real retweet_count;
    real trade_topic;
    real china_topic;
    real financial_topic;
    real labor_topic;
    real industry_topic;
    real currency_rates_topic;
    real general_related_topic;
    real budget_topic;
    real foreign_policy_topic;
    real covid_topic;
    real sensitive_topic;
    real environment_topic;
    real sentiment;
    real<lower=0> sigma;
}
model {
    y ~ normal(alpha + favorite_count * x + retweet_count * x1 + trade_topic * x2 + china_topic * x3 + financial_topic * x4 + labor_topic * x5 + industry_topic * x6 + currency_rates_topic * x7 + general_related_topic * x8 + budget_topic * x9 + foreign_policy_topic * x10 + covid_topic * x11 + sensitive_topic * x12 + environment_topic * x13 + sentiment * x14, sigma);
}
"""

In [286]:
# implement the model using Pystan

data = {'N': len(inspect_df), 'x': inspect_df.iloc[:,0], 'x1': inspect_df.iloc[:,1], 'x2': inspect_df.iloc[:,2], 
        'x3': inspect_df.iloc[:,3], 'x4': inspect_df.iloc[:,4], 'x5': inspect_df.iloc[:,5], 'x6': inspect_df.iloc[:,6], 
        'x7': inspect_df.iloc[:,7], 'x8': inspect_df.iloc[:,8], 'x9': inspect_df.iloc[:,9], 'x10': inspect_df.iloc[:,10], 
        'x11': inspect_df.iloc[:,11], 'x12': inspect_df.iloc[:,12], 'x13': inspect_df.iloc[:,13], 
        'x14': inspect_df.iloc[:,14], 'y': inspect_df.iloc[:,-1]}

# Compile the model
smod = pystan.StanModel(model_code=model)

# Train the model and generate samples
fit = smod.sampling(data=data, iter=1000, chains=4, warmup=500, thin=1, seed=101)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4cd63eaaa8fafd9a7c8d59c85037c311 NOW.


In [287]:
summary_dict = fit.summary()
df_summary = pd.DataFrame(summary_dict['summary'], 
                  columns=summary_dict['summary_colnames'], 
                  index=summary_dict['summary_rownames'])

In [288]:
# Pystan results
df_summary

,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
alpha,0.000024,8.491863e-07,0.000037,-0.000047,-0.000002,0.000024,0.000048,0.000096,1869.710766,1.000081
favorite_count,-0.000277,4.486539e-06,0.000216,-0.000701,-0.000424,-0.000277,-0.000131,0.000160,2307.262346,0.998727
retweet_count,0.000221,4.197204e-06,0.000201,-0.000187,0.000084,0.000217,0.000359,0.000623,2300.232873,0.998794
trade_topic,0.000299,4.617007e-06,0.000206,-0.000114,0.000161,0.000300,0.000440,0.000684,1998.591570,0.998710
china_topic,-0.000884,1.549593e-05,0.000630,-0.002069,-0.001333,-0.000910,-0.000438,0.000350,1655.329826,0.999737
financial_topic,0.000729,1.025253e-05,0.000460,-0.000185,0.000413,0.000723,0.001043,0.001645,2008.729790,1.000972
labor_topic,-0.000109,2.259581e-06,0.000104,-0.000312,-0.000180,-0.000109,-0.000034,0.000091,2123.989115,0.999954
industry_topic,0.000340,5.434305e-06,0.000273,-0.000187,0.000165,0.000339,0.000520,0.000870,2515.135986,1.001618
currency_rates_topic,0.000068,4.572054e-06,0.000221,-0.000376,-0.000078,0.000068,0.000210,0.000498,2339.329576,1.000194
general_related_topic,-0.000178,2.817601e-06,0.000131,-0.000434,-0.000268,-0.000177,-0.000086,0.000082,2159.102140,0.999093


In [289]:
df_summary.to_csv('mcmc_pence_dow_price_after_2016.csv')

In [290]:
probs = pd.DataFrame()
columns = ['favorite_count', 'retweet_count', 'trade_topic', 'china_topic',
       'financial_topic', 'labor_topic', 'industry_topic',
       'currency_rates_topic', 'general_related_topic', 'budget_topic',
       'foreign_policy_topic', 'covid_topic', 'sensitive_topic',
       'environment_topic', 'sentiment']
for column in columns:
    probs[column] = fit[column]

In [291]:
probs.to_csv('extended_mcmc_pence_dow_price_after_2016.csv')

# Generalization

In [ ]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.00000000001,normalize=False)
rr.fit(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1])
w = rr.coef_

In [ ]:
print(rr.score(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1]))

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.000000001,normalize=False)
clf.fit(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1])
print(clf.score(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1]))

In [ ]:
coefs=pd.DataFrame(columns=['variable','coefficients'])

In [ ]:
coefs['variable']=inspect_df.iloc[:,:-1].columns

In [ ]:
coefs['coefficients'] = clf.coef_

In [ ]:
len(coefs[coefs['coefficients']!=0])

In [ ]:
for i,x in coefs[coefs['coefficients']!=0].sort_values('coefficients').iterrows():
    print(x['variable'])